
---

# 📈 **15. Monitoring & Logging**

---

## 🧪 Quick metrics (built-in)

```bash
docker stats --no-stream          # one-shot CPU/RAM/NET/IO
docker top <name>                 # processes in container
docker inspect <name>             # JSON: limits, health, mounts, IP
```

---

## 📝 Runtime logs (built-in)

```bash
docker logs -f --tail 100 <name>  # follow recent logs
```

**Rotate logs**

```bash
docker run --log-driver local \
  --log-opt max-size=10m --log-opt max-file=3 myapp
# or json-file driver:
# --log-driver json-file --log-opt max-size=10m --log-opt max-file=3
```

> Linux raw files: `/var/lib/docker/containers/<id>/*-json.log`

---

## 📈 Prometheus + Grafana (metrics)

**cAdvisor (container metrics exporter)**

```bash
docker run -d --name=cadvisor -p 8080:8080 \
  -v /:/rootfs:ro -v /var/run:/var/run:rw \
  -v /sys:/sys:ro -v /var/lib/docker/:/var/lib/docker:ro \
  gcr.io/cadvisor/cadvisor
```

**Prometheus scrape**

```yaml
scrape_configs:
  - job_name: cadvisor
    static_configs: [{ targets: ["localhost:8080"] }]
```

Then add **Prometheus** to **Grafana** and import a Docker/cAdvisor dashboard.

---

## 🪵 Centralized logs (ELK/EFK/Loki)

**Driver → collector**

```bash
docker run --log-driver fluentd \
  --log-opt fluentd-address=localhost:24224 myapp
```

**File shipper**

* Keep default logs; ship with **Fluent Bit/Filebeat** from
  `/var/lib/docker/containers/*/*.log` → **Elasticsearch** (Kibana) or **Loki** (Grafana).

---

## 🩺 Events & health

```bash
docker events                                  # live daemon events
docker inspect -f '{{.State.Health.Status}}' <name>
```

**Dockerfile probe**

```dockerfile
HEALTHCHECK CMD curl -fsS http://localhost/health || exit 1
```

---

## ✅ Best practices (1-liners)

* Use **`docker stats`** for quick checks; **Prometheus+cAdvisor** for dashboards.
* Prefer **`local`/`json-file`** with **rotation** to cap disk.
* Keep **metrics ≠ logs** (Prometheus for metrics; ELK/Loki for logs).
* Avoid logging secrets; scrub at app/collector.
* Name containers consistently; add labels for filtering.

---
